In [1]:
import os
import sys

# sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
# os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
sys.path.append('/home/ec2-user/SageMaker/PETsARD')
os.chdir('/home/ec2-user/SageMaker/PETsARD/demo')
!pip install -r '../requirements.txt'

import PETsARD

Ignoring appnope: markers 'python_version >= "3.10" and python_version < "3.11" and platform_system == "Darwin"' don't match your environment
Ignoring cffi: markers 'python_version >= "3.10" and python_version < "3.11" and implementation_name == "pypy"' don't match your environment
Ignoring colorama: markers 'python_version >= "3.10" and python_version < "3.11" and (sys_platform == "win32" or platform_system == "Windows")' don't match your environment
Ignoring pycparser: markers 'python_version >= "3.10" and python_version < "3.11" and implementation_name == "pypy"' don't match your environment
Ignoring pywin32: markers 'sys_platform == "win32" and platform_python_implementation != "PyPy" and python_version >= "3.10" and python_version < "3.11"' don't match your environment


In [4]:
import pandas as pd
import numpy as np

first_result = True


exp = 'PETsARD[20231227-094238]'
_eval_module = 'SDMetrics'
filename_rpt = f"__{exp}_Report_{_eval_module}.csv"

preproc = 1

for split in range(1,10+1):
    filepath_ori = f"{exp}_Trial[{str(split).zfill(2)}][Ori].csv"
    data_ori = PETsARD.Loader(filepath_ori).data

    for syn in range(1,4+1):
        str_split = f"{str(split).zfill(2)}-{preproc}-{syn}"
        filepath_syn = f"{exp}_Trial[{str_split}]Postproc.csv"
        data_syn = PETsARD.Loader(filepath_syn).data



        eval_diagnostic = PETsARD.Evaluator(
            evaluating_method='sdmetrics-single_table-diagnosticreport',
            data={
                'ori': data_ori,
                'syn': data_syn
            }
        )
        eval_diagnostic.eval()

        df_diagnostic = pd.DataFrame()
        df_diagnostic['Properties'] = ['DiagnostReport']
        df_diagnostic['Score'] = eval_diagnostic.Evaluator.evaluation['score']
        df_diagnostic['Metric'] = 'score values'
        df_diagnostic[['Column', 'Column 2', 'Real Correlation', 'Synthetic Correlation']] = pd.NA
        df_diagnostic = df_diagnostic[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_diagnostic_prop = pd.DataFrame.from_dict(
            eval_diagnostic.Evaluator.evaluation['properties'],
            orient='index'
        ).reset_index(drop=False).rename(columns={'index': 'Properties'})
        df_diagnostic_prop['Metric'] = 'properties values'
        df_diagnostic_prop[['Column', 'Column 2', 'Real Correlation', 'Synthetic Correlation']] = pd.NA
        df_diagnostic_prop = df_diagnostic_prop[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_dv = eval_diagnostic.Evaluator.evaluation['details']['Data Validity']
        df_dv['Properties'] = 'Data Structure'
        df_dv['Column 2'] = pd.NA
        df_dv['Real Correlation'] = pd.NA
        df_dv['Synthetic Correlation'] = pd.NA
        df_dv = df_dv[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_ds = eval_diagnostic.Evaluator.evaluation['details']['Data Structure']
        df_ds['Properties'] = 'Data Structure'
        df_ds['Column'] = '-'
        df_ds['Column 2'] = pd.NA
        df_ds['Real Correlation'] = pd.NA
        df_ds['Synthetic Correlation'] = pd.NA
        df_ds = df_ds[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]



        eval_quality = PETsARD.Evaluator(
            evaluating_method='sdmetrics-single_table-qualityreport',
            data={
                'ori': data_ori,
                'syn': data_syn
            }
        )
        eval_quality.eval()

        df_quality = pd.DataFrame()
        df_quality['Properties'] = ['QualityReport']
        df_quality['Score'] = eval_quality.Evaluator.evaluation['score']
        df_quality['Metric'] = 'score values'
        df_quality[['Column', 'Column 2', 'Real Correlation', 'Synthetic Correlation']] = pd.NA
        df_quality = df_quality[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_quality_prop = pd.DataFrame.from_dict(
            eval_quality.Evaluator.evaluation['properties'],
            orient='index'
        ).reset_index(drop=False).rename(columns={'index': 'Properties'})
        df_quality_prop['Metric'] = 'properties values'
        df_quality_prop[['Column', 'Column 2', 'Real Correlation', 'Synthetic Correlation']] = pd.NA
        df_quality_prop = df_quality_prop[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_cs = eval_quality.Evaluator.evaluation['details']['Column Shapes']
        df_cs['Properties'] = 'Column Shapes'
        df_cs['Column 2'] = pd.NA
        df_cs['Real Correlation'] = pd.NA
        df_cs['Synthetic Correlation'] = pd.NA
        df_cs = df_cs[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_cpt = eval_quality.Evaluator.evaluation['details']['Column Pair Trends']
        df_cpt['Properties'] = 'Column Pair Trends'
        df_cpt.rename(columns={"Column 1": "Column"}, inplace=True)
        df_cpt['Column 2'] = pd.NA
        df_cpt = df_cpt[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]



        df_result = pd.concat([
            df_diagnostic, df_diagnostic_prop, df_dv, df_ds,
            df_quality, df_quality_prop, df_cs, df_cpt
        ]).reset_index(drop=True)



        df_final = pd.DataFrame()
        df_final['Properties'] = ['Final']
        df_final['Score'] = np.mean(df_result.loc[df_result['Properties'].isin(['DiagnostReport', 'QualityReport'])]['Score'])
        df_final['Metric'] = 'score values'
        df_final[['Column', 'Column 2', 'Real Correlation', 'Synthetic Correlation']] = pd.NA
        df_final = df_final[['Properties', 'Column', 'Column 2', 'Metric', 'Score', 'Real Correlation', 'Synthetic Correlation']]

        df_result = pd.concat([
            df_result, df_final
        ]).reset_index(drop=True)
        df_result['exp'] = exp
        df_result['split'] = str_split
        df_result['eval_module'] = 'SDMetrics'
        df_result = df_result[[
            'exp', 'split', 'eval_module',
            'Properties', 'Column', 'Column 2',
            'Metric', 'Score',
            'Real Correlation', 'Synthetic Correlation'
        ]]

        with open(filename_rpt, 'a') as f:
            if first_result:
                csv_string = df_result.to_csv(index=False, sep=',')
                first_result = False
            else:
                csv_string = df_result.to_csv(index=False, sep=',', header=False)

            f.write(csv_string)
        print(f"{filepath_ori} + {filepath_syn} is done.")

Evaluator (SDMetrics): Evaluating DiagnosticReport.
Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 625.74it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%
Evaluator (SDMetrics): Evaluating DiagnosticReport spent 0.0135 sec.
Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 66/66 [00:01<00:00, 60.83it/s]

Overall Score: 85.92%

Properties:
- Column Shapes: 90.96%
- Column Pair Trends: 80.87%
Evaluator (SDMetrics): Evaluating QualityReport spent 1.1031 sec.
PETsARD[20231227-094238]_Trial[01][Ori].csv + PETsARD[20231227-094238]_Trial[01-1-1]Postproc.csv is done.
Evaluator (SDMetrics): Evaluating DiagnosticReport.
Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 630.82it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%
Evaluator (SDMetrics): Evaluat